# 뉴스 크롤링 자연어 처리 함수
* 입력 종목코드로 NAVER 뉴스 크롤링
* 해당 뉴스 제목들 중 부정적인 단어LIST가 들어있는 DF행 제거 후 개수 반환 함수

In [ ]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

In [ ]:
import os
import time
import re

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
#마지막 페이지 번호
def getLastPageNum(soup) :
    return int(soup.find("table", class_="Nnavi").find("td", class_="pgRR").find("a").get("href").split("&")[1].split("=")[1])

In [ ]:
#뉴스 데이터 가져오기(관련기사는 제거)
def getPageNewsData(news_url, page) :
    url = news_url + "&page=" + str(page)
    response = requests.get(url)
    response.encoding = 'euc-kr' #한글깨짐방지
    soup = BeautifulSoup(response.text, 'html.parser')
    
    table = soup.find('table')
    rlist = table.find_all('tr', class_="relation_lst")
    for tr in rlist :
        tr.decompose()

    df = pd.read_html(str(table), header=0)[0].dropna()
    return df

In [ ]:
def GetIssueCnt(code, startDate, bad_word) :
  #기사 가져오기
  news_url = 'https://finance.naver.com/item/news_news.nhn?code='+code # 뉴스 기사
  response = requests.get(news_url)
  soup = BeautifulSoup(response.text, 'html.parser')

  last_page = getLastPageNum(soup)
  last_index = last_page+1

  news_df = getPageNewsData(news_url, 1).dropna()
  for i in range(2, last_index) :
      tmp_df = getPageNewsData(news_url, i)
      add_df = tmp_df[tmp_df['날짜'] > startDate]
      news_df = pd.concat([news_df, add_df], ignore_index=True)
      if len(add_df) < len(tmp_df) : 
          break

  news_df = news_df.drop_duplicates()
  news_df = news_df.drop(['정보제공', '날짜'], axis=1).rename({'제목':'subject'}, axis='columns')

  list = []
  for i in range(0, len(news_df)) :
      index = news_df["subject"].index[i]
      list.append(" ".join(clean(news_df["subject"][index])))
      #list.append(" ".join(news_df["subject"][index]))
  df= pd.DataFrame(list)
  df.columns = ["subject"]
  df['subject'] = df['subject'].astype(str)
  df = df[~df['subject'].str.contains('|'.join(bad_word))] #bad_word 에 포함된 행을 제거 
  return df

In [ ]:
def clean(sub) :
    text = re.sub("[^ㄱ-ㅣ가-힣]", " ", sub)
    text2 = text.split()
    text3 = [w for w in text2 if not w in stopwords]
    return text3

In [ ]:
company = input("회사명 or 종목명 입력: 씨젠 디폴트 ")
#code = convert_to_code(company)
code = '096530'

if code is None or code == '' :
    print("코드입력 없으면 씨젠 096530 검색")
    code = '096530'
    
startDate = input("뉴스 기사 마지막 날짜: ")

if startDate is None or startDate == '' :
    print("날짜 입력이 없어 Default(2020.06.30)를 수행합니다.")
    startDate = '2020.06.30'

#부정적인 단어는 빼고 이슈 기사의 갯수를 가져온다.
bad_word = ['하락세', '사기', '실망', '적발','부진','약세', '고소' ,'신고', '우려' ,'불능' ,'부실' ,'희박', '연기']
stopwords = [company] #입력받은 회사명은 불용어 제거 -> 딱히 필요없음
df =GetIssueCnt(code, startDate, bad_word)
df
#len(df) # 뉴스제목에 부정적인 단어가 들어간 기사를 뺀 개수





회사명 or 종목명 입력: 씨젠 디폴트 
뉴스 기사 마지막 날짜: 
날짜 입력이 없어 Default(2020.06.30)를 수행합니다.


,subject
0,씨젠 임직원 명에서 명으로 일자리 유공 표창 수상
1,천종윤 씨젠 대표 보건의료기술 진흥 대통령 표창 수상
2,겨울 보너스 배당주 막차 타볼까 씨젠 주당 원 배당
3,씨젠 제조 구매 총괄 부사장에 이기선 대림산업 영입
4,씨젠 타액 진단 코로나 진단 키트 개발 기대 하나금투
...,...
214,씨젠 개인 투자자 관심 집중 주가
215,일 기관 코스닥에서 에스엠 씨젠
216,주식 초고수는 지금 셀트리온 사자 위 씨젠은 집중매도
217,씨젠 검색 상위 랭킹 주가
